In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 460kB 38.0MB/s 
     |████████████████████████████████| 92kB 8.7MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [16]:
from tqdm import tqdm
from konlpy import tag
from collections import defaultdict, Counter
import math

데이터를 가져와서 token화 + vocab만들기

In [3]:
train_data = [
  "정말 맛있습니다. 추천합니다.",
  "기대했던 것보단 별로였네요.",
  "다 좋은데 가격이 너무 비싸서 다시 가고 싶다는 생각이 안 드네요.",
  "완전 최고입니다! 재방문 의사 있습니다.",
  "음식도 서비스도 다 만족스러웠습니다.",
  "위생 상태가 좀 별로였습니다. 좀 더 개선되기를 바랍니다.",
  "맛도 좋았고 직원분들 서비스도 너무 친절했습니다.",
  "기념일에 방문했는데 음식도 분위기도 서비스도 다 좋았습니다.",
  "전반적으로 음식이 너무 짰습니다. 저는 별로였네요.",
  "위생에 조금 더 신경 썼으면 좋겠습니다. 조금 불쾌했습니다."
]
train_labels = [1, 0, 0, 1, 1, 0, 1, 1, 0, 0]

test_data = [
  "정말 좋았습니다. 또 가고 싶네요.",
  "별로였습니다. 되도록 가지 마세요.",
  "다른 분들께도 추천드릴 수 있을 만큼 만족했습니다.",
  "서비스가 좀 더 개선되었으면 좋겠습니다. 기분이 좀 나빴습니다."
]

In [6]:
tokenizer = tag.Okt()

In [8]:
def tokenization(data):
  data_tokenized = []
  for i in data:
    tokens = tokenizer.morphs(i)
    data_tokenized.append(tokens)
  return data_tokenized

In [10]:
train_token = tokenization(train_data)
test_token = tokenization(test_data)

In [ ]:
word_count = defaultdict()
for i in train_token:
  for j in i:
    if j in word_count:
      word_count[j] += 1
    else:
      word_count[j] = 1
word_count

In [ ]:
w2i = defaultdict()
for i in word_count:
  if i not in w2i:
    w2i[i] = len(w2i)
w2i

In [15]:
len(w2i)

66

In [83]:
class NBC():
  def __init__(self,w2i,train_token,test_token,train_labels,k=0.1):
    self.prior_list = defaultdict()
    self.likelihood_list = defaultdict()
    self.train_token = train_token
    self.test_token = test_token
    self.train_labels = train_labels
    self.k = k
    self.w2i = w2i

  def train(self):
    self.priors()
    self.likelihoods()
  def reference(self):
    result = []
    for tokens in test_token:
      prob_0,prob_1 = 0,0
      for token in tokens:
        if token in self.likelihood_list:
          prob_0 += math.log(self.likelihood_list[token][0])
          prob_1 += math.log(self.likelihood_list[token][1])
      prob_0 += math.log(self.prior_list[0])
      prob_1 += math.log(self.prior_list[1])
      result.append(0 if prob_0 > prob_1 else 1)
    return result

  def priors(self):
    for i in train_labels:
      if i in self.prior_list:
        self.prior_list[i] += 1
      else:
        self.prior_list[i] = 1
    for j in range(len(self.prior_list)):
      self.prior_list[i] /= len(train_labels)

  def likelihoods(self):
    count_0,count_1 = 0,0
    for i, tokens in enumerate(self.train_token):
      if train_labels[i] == 0:
        count_0 += len(tokens)
      else:
        count_1 += len(tokens)
      for token in tokens:
        if token in self.likelihood_list:
          self.likelihood_list[token][train_labels[i]] += 1
        else:
          self.likelihood_list[token] ={0:0,1:0}
          self.likelihood_list[token][train_labels[i]] += 1
    for j in self.likelihood_list:
      self.likelihood_list[j][0] = (self.likelihood_list[j][0]+self.k)/ (count_0+len(self.w2i)*self.k)
      self.likelihood_list[j][1] = (self.likelihood_list[j][1]+self.k)/ (count_1+len(self.w2i)*self.k)


In [84]:
nbc = NBC(w2i,train_token,test_token,train_labels,k=0.1)

In [85]:
nbc.train()

In [86]:
nbc.likelihood_list

defaultdict(None,
            {'!': {0: 0.0015243902439024393, 1: 0.02131782945736434},
             '.': {0: 0.12347560975609756, 1: 0.11821705426356588},
             '가': {0: 0.016768292682926834, 1: 0.001937984496124031},
             '가격': {0: 0.016768292682926834, 1: 0.001937984496124031},
             '가고': {0: 0.016768292682926834, 1: 0.001937984496124031},
             '개선': {0: 0.016768292682926834, 1: 0.001937984496124031},
             '것': {0: 0.016768292682926834, 1: 0.001937984496124031},
             '기념일': {0: 0.0015243902439024393, 1: 0.02131782945736434},
             '기대했던': {0: 0.016768292682926834, 1: 0.001937984496124031},
             '기를': {0: 0.016768292682926834, 1: 0.001937984496124031},
             '너무': {0: 0.03201219512195122, 1: 0.02131782945736434},
             '는': {0: 0.016768292682926834, 1: 0.001937984496124031},
             '다': {0: 0.016768292682926834, 1: 0.040697674418604654},
             '다시': {0: 0.016768292682926834, 1: 0.0019379844961240

In [87]:
nbc.reference()

[1, 0, 1, 0]